In [ ]:
# Run this cell first to import some modules
# we'll need to run the examples below.
%matplotlib inline
import math
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML

from pydrake.all import (AddMultibodyPlantSceneGraph,
                         DiagramBuilder,
                         Parser,
                         Simulator,
                         SignalLogger)
from underactuated import ( FindResource, PlanarSceneGraphVisualizer )


In [ ]:
# This cell sets up a double pendulum, runs a simulation, and
# renders a video of its results.

# Set up a block diagram with the robot (dynamics) and a visualization block.
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder)

# Load the double pendulum from Universal Robot Description Format
parser = Parser(plant, scene_graph)
parser.AddModelFromFile(FindResource("double_pendulum/double_pendulum.urdf"))
plant.Finalize()

builder.ExportInput(plant.get_actuation_input_port())
visualizer = builder.AddSystem(PlanarSceneGraphVisualizer(scene_graph,
                                                          xlim=[-2.8, 2.8],
                                                          ylim=[-2.8, 2.8]))
builder.Connect(scene_graph.get_pose_bundle_output_port(),
                visualizer.get_input_port(0))

# The logger needs to be told to expect a 4-element input
# (the 4-element double pendulum state, in this case).
logger = builder.AddSystem(SignalLogger(4))
logger.DeclarePeriodicPublish(0.033333, 0.0)
builder.Connect(plant.get_state_output_port(), logger.get_input_port(0))

diagram = builder.Build()

# Set up a simulator to run this diagram
simulator = Simulator(diagram)

# Set the initial conditions
context = simulator.get_mutable_context()
context.SetContinuousState([1., 1., 0., 0.])  # (theta1, theta2, theta1dot, theta2dot)
context.FixInputPort(0, [0.,0.])   # Zero input torques

# Simulate and animate
visualizer.start_recording(show=False)
simulator.StepTo(10.0)
ani = visualizer.get_recording(repeat=True)
HTML(ani.to_html5_video())

In [ ]:
# Run this cell after running a simulation to plot traces
# of the pendulum state across time.
fields = ["shoulder", "elbow"]
for i in range(2):
    #plt.subplot(3, 1, i+1)
    plt.plot(logger.sample_times(), logger.data()[(i, i+2), :].transpose())
    plt.legend(["position", "velocity"])
    plt.xlabel('t')
    plt.ylabel(fields[i])
    plt.grid(True)
    plt.show()